<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/haskell_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# メモ

Haskell で 行列 matrix を学ぶ。


行列 haskell  
で検索

1. ARRAY PROGRAMMING IN HASKELL  
https://www.tweag.io/blog/2017-08-09-array-programming-in-haskell/  
1. humatrix tutorial old version   
http://dis.um.es/profesores/alberto/material/hmatrix.pdf  
1. hmatrixの使い方とニューラルネットワークの実装例  
https://qiita.com/lotz/items/2c932b45f78f6fc70e9c
1. Haskellの線形代数用ライブラリで多変量ガウス分布を計算した際のメモ  
https://mmitou.hatenadiary.org/entry/20121021/1350798725
1. ENTER THE MATRIX, HASKELL STYLE  
https://www.tweag.io/blog/2017-08-31-hmatrix/
1. Dimensions and Haskell: Introduction
https://serokell.io/blog/dimensions-and-haskell-introduction
1. 行列計算用のHaskellライブラリ hmatrix を入れる  
https://mano.xyz/391/
1. hmatrix-0.20.1: Numeric Linear Algebra
1. haskell プログラム集  
https://userweb.mnet.ne.jp/tnomura/examples/matrix.html
1. Haskellで数値計算 線形代数編  
https://lqtmirage.hatenablog.com/entry/2018/07/12/232852
1. Haskellの行列計算 - 保存と読み出し、複素行列について  
https://qiita.com/tkch_pe/items/d6cfbc333d6c60751184
1. Haskell で行列の計算をしてみる  
http://tsumuji.cocolog-nifty.com/tsumuji/2011/02/haskell-0b4a.html

In [1]:
# colab の環境に haskell と hmatrix を入れる
# まず haskell をインストールする
%%capture
!apt install haskell-platform

In [4]:
# 次に hmatrix を入れる
%%capture
!sudo cabal update
!sudo cabal install hmatrix

Resolving dependencies...
Configuring semigroups-0.20...
Configuring base-orphans-0.8.6...
Building base-orphans-0.8.6...
Building semigroups-0.20...
Installed semigroups-0.20
Installed base-orphans-0.8.6
Configuring storable-complex-0.2.3.0...
Building storable-complex-0.2.3.0...
Installed storable-complex-0.2.3.0
Configuring hmatrix-0.20.2...
Building hmatrix-0.20.2...
Installed hmatrix-0.20.2


In [5]:
# 次のコードでエラーが出なければ hmatrix がインストールされている
!ghc -e $'import Numeric.LinearAlgebra'

In [ ]:
# cabal でインストールできなかった場合、手動で入れることを試みる
# haskell の線形代数パッケージ hmatrix は Fortran 時代から有名な
# BLAS (Basic Linear Algebra Subprograms)
# LAPACK — Linear Algebra PACKage
# を使っている
# !apt install libghc6-hunit-dev libghc6-quickcheck-dev
# !apt install libgsl0-dev libblas-dev liblapack-dev libatlas-base-dev

In [ ]:
# storable-complex は apt で入れようとするとパッケージがない、と言われる
# !wget http://hackage.haskell.org/packages/archive/storable-complex/0.2/storable-complex-0.2.tar.gz
# !tar -xzf storable-complex-0.2.tar.gz
# !cd storable-complex-0.2/; runhaskell Setup configure; cabal install

# いまここ

In [6]:
# hmatrix の核となるのは次の二つのデータタイプである
# data Vector e
# data Matrix e
%%writefile temp.hs
import Numeric.LinearAlgebra
main = do
    print (vector [1,2,3] * vector [3,0,-2])

Writing temp.hs


In [7]:
!runghc temp.hs

[3.0,0.0,-6.0]


In [ ]:
# 見かけは list だが、型は vector である
# hmatrix> :t vector [1,2,3] * vector [3,0,-2]
# vector [1,2,3] * vector [3,0,-2] :: Vector R

# 要素の型の R は hmatrix の中の数値の型の synonym である
# type R = Double
# type C = Complex Double
# type I = CInt
# type Z = Int64

In [ ]:
# 以上が vector だが、行列 matrix でも同様に
%%writefile matrix01.hs
import Numeric.LinearAlgebra
main = do
    print (matrix 3 [1..9] * ident 3)


Writing matrix01.hs


In [ ]:
!runghc matrix01.hs

(3><3)
 [ 1.0, 0.0, 0.0
 , 0.0, 5.0, 0.0
 , 0.0, 0.0, 9.0 ]


In [ ]:
# hmatrix> :t matrix 3 [1..9] * ident 3
# matrix 3 [1..9] * ident 3 :: Matrix R

# matrix 3 は後続するリストを 3列の行列にする
# ident は 3 x 3 の単位行列を作る

In [ ]:
%%writefile matrix02.hs
import Numeric.LinearAlgebra
main = do
    print (matrix 3 [1..15] )

Overwriting matrix02.hs


In [ ]:
!runghc matrix02.hs

(5><3)
 [  1.0,  2.0,  3.0
 ,  4.0,  5.0,  6.0
 ,  7.0,  8.0,  9.0
 , 10.0, 11.0, 12.0
 , 13.0, 14.0, 15.0 ]


In [ ]:
%%writefile matrix03.hs
import Numeric.LinearAlgebra
main = do
    print ((ident 3 ):: Matrix I)

Overwriting matrix03.hs


In [ ]:
!runghc matrix03.hs

(3><3)
 [ 1, 0, 0
 , 0, 1, 0
 , 0, 0, 1 ]


In [ ]:
# GENERALISED MATRICES
# 説明がわからないので、以降例だけをしめします
# 上記の Matrix型と Vector型に加えて GMatrix という型がある
# 次の例は疎 sparse な、2x2000 の行列でゼロでない要素が 2つだけのものを
# compressed sparse row (CSR) format で示している
%%writefile sparse01.hs
import Numeric.LinearAlgebra
main = 
   print (mkSparse [((0,999),1.0),((1,1999),2.0)])


Writing sparse01.hs


In [ ]:
!runghc sparse01.hs

SparseR {gmCSR = CSR {csrVals = [1.0,2.0], csrCols = [1000,2000], csrRows = [1,2,3], csrNRows = 2, csrNCols = 2000}, nRows = 2, nCols = 2000}


```
SparseR
{ gmCSR = CSR
          { csrVals = [1.0,2.0]
          , csrCols = [1000,2000]
          , csrRows = [1,2,3]
          , csrNRows = 2
          , csrNCols = 2000
          }
, nRows = 2
, nCols = 2000
}
```

In [ ]:
# f [x,y] = 10*(x-1)^2 + 20*(y-2)^2 + 30
# の最小値を gradient を用いずに見つける
%%script false
%%writefile minima.hs
import Numeric.LinearAlgebra
minimizeS :: ([Double] -> Double) -> [Double] -> ([Double], Matrix Double)
minimizeS f xi
  = minimize NMSimplex2 1E-2 100 (replicate (length xi) 1) f xi


In [ ]:
%%script false
%%writefile minimize.hs
-- the multidimensional minimization example in the GSL manual
import Numeric.GSL
import Numeric.LinearAlgebra
import Graphics.Plot
import Text.Printf(printf)

-- the function to be minimized
f [x,y] = 10*(x-1)^2 + 20*(y-2)^2 + 30

-- exact gradient
df [x,y] = [20*(x-1), 40*(y-2)]

-- a minimization algorithm which does not require the gradient
minimizeS f xi = minimize NMSimplex2 1E-2 100 (replicate (length xi) 1) f xi

-- Numerical estimation of the gradient
gradient f v = [partialDerivative k f v | k <- [0 .. length v -1]]

partialDerivative n f v = fst (derivCentral 0.01 g (v!!n)) where
    g x = f (concat [a,x:b])
    (a,_:b) = splitAt n v

disp' = putStrLn . format "  " (printf "%.3f")

allMethods :: (Enum a, Bounded a) => [a]
allMethods = [minBound .. maxBound]

test method = do
    print method
    let (s,p) = minimize method 1E-2 30 [1,1] f [5,7]
    print s
    disp' p

testD method = do
    print method
    let (s,p) = minimizeD method 1E-3 30 1E-2 1E-4 f df [5,7]
    print s
    disp' p

testD' method = do
    putStrLn $ show method ++ " with estimated gradient"
    let (s,p) = minimizeD method 1E-3 30 1E-2 1E-4 f (gradient f) [5,7]
    print s
    disp' p

main = do
    mapM_ test [NMSimplex, NMSimplex2]
    mapM_ testD allMethods
    testD' ConjugateFR
    mplot $ drop 3 . toColumns . snd $ minimizeS f [5,7]

In [ ]:
# 標準モジュール Data.Array 含まれる配列は
# immutable, boxed, non-strict であり、これを使って配列のアルゴリズムを作ることができる。
# しかし、boxing や non-strictness は連想配列に依存し、haskell compiler の最適化を
# 疎外し、結果として遅くなってしまう。


In [ ]:
# non-strictness の恩恵としての表現の豊かさを A Gentle Introduction to Haskell にある
# 次の例で見てみよう。

The wavefront function defines

an nxn array whose leftmost column and topmost row are populated
with 1s (by the first two list comprehensions below). All other
array elements are defined as the sum of the three elements
to their left, top, and top-left (by the third list comprehension
in the definition).

wavefront :: Int -> Array (Int,Int) Int
wavefront n = arr
 where
   arr = array ((1,1),(n,n))
               ([((1,j), 1) | j <- [1..n]] ++
                [((i,1), 1) | i <- [2..n]] ++
                [((i,j), arr!(i,j-1) + arr!(i-1,j-1) + arr!
(i-1,j))
                            | i <- [2..n], j <- [2..n]])
The ! infix operator implements array indexing:

(!) :: Ix i => Array i e -> i -> e
Moreover, the function array constructs an array from an association
list mapping indexes to values:

array :: Ix i => (i, i) -> [(i, e)] -> Array i e
The elegance of wavefront is in the recursive definition of the
array arr.
In the expression arr!(i,j-1) + arr!(i-1,j-1) + arr
!(i-1,j), we access the elements to the left, top, and top-left
of the current one by appropriate indexing of the very array
that we are currently in the process of defining.
Such a recursive
dependency is only valid for a non-strict data structure.

BOXING
Unfortunately, the expressiveness of non-strict arrays comes
at a price, especially if the array elements are simple numbers.
Instead of being able to store those numeric elements in-place
in the array, non-strict arrays require a boxed representation,
where the elements are pointers to heap objects containing the
numeric values.
This additional indirection requires extra memory
and drastically reduces the efficiency of array access, especially
in tight loops.
The layout difference between an unboxed (left)
and a boxed (right) representation is illustrated below.

boxed versus unboxed array representation

While both strict and non-strict data structures admit boxed
representations, non-strict structures typically require boxing.
To provide an alternative to the standard non-strict arrays,
the array package provides strict, unboxed arrays of type Data.Array.Unboxed.UArray
i e.
By way of overloading via the type class Data.Array.IArray.IArray,
they provide the same API as the standard non-strict, boxed
arrays.
However, the element type is restricted to basic types
that can be stored unboxed, such as integral and floating-point
numeric types.

Unfortunately, array construction based on association lists,
such as the array function, still severely limits the performance
of immutable UArrays.
Nevertheless, at least, array access by
way of (!) is efficient for unboxed arrays.

IMMUTABILITY
While immutable arrays —i.e., arrays that cannot directly be
in-place updated— are semantically simpler, it turns out that
indexed-based array construction is drastically more efficient
for mutable arrays.
Hence, computationally demanding Haskell
array code typically adopts a two-phase array life cycle: (1)
arrays are allocated as mutable arrays and initialised using
in-place array update; once initialised, (2) they are frozen
by making them immutable.

This usage pattern is supported by the interface provided by
the Data.Array.MArray.MArray class and we use it in the following
example function generate to initialise an array of Doubles
with an index-based generator function gen:

generate :: Ix i => (i, i) -> (i -> Double) -> UArray i Double
generate bnds gen
 = runSTUArray $ do
     arr <- newArray_ bnds
     mapM_ (\i -> writeArray arr i (gen i)) (range bnds)
     return arr
Mutable arrays come in various flavours that are, in particularly,
distinguished by the monad in which the array operations take
place.
Usually, this is either IO or ST, and the array package
provides both boxed and unboxed variants for both monads.
We
have the boxed IOArray and the unboxed IOUArray as well as the
boxed STArray and the unboxed STUArray.
The above definition
of generate uses STUArray to initialise the array, and then,
freezes it into a UArray, which is returned.
The choice of STUArray
is implicit in the use of runSTUArray, which executes the code
in the state transformer monad ST and freezes the STUArray into
a UArray.

The function newArray_ provides a fresh uninitialised array:
newArray_ :: (MArray a e m, Ix i) => (i, i) -> m (a i e)
We can read and write a mutable array with

readArray  :: (MArray a e m, Ix i) => a i e -> i      -> m e
writeArray :: (MArray a e m, Ix i) => a i e -> i -> e -> m ()
In the definition of generate, we use writeArray once on each
index in the range range bnds of the mutable array to initialise
the value at index i with the value of the generator function
at that index, gen i.

Generally, we can freeze a mutable array, obtaining an immutable
array, with

freeze :: (Ix i, MArray a e m, IArray b e) => a i e -> m (b i
e)
There is also the unsafe variant unsafeFreeze that avoids copying
the array during freezing, but puts the onus on the programmer
to ensure that the mutable argument is subsequently not updated
anymore.
In the code for generate above, we indirectly use unsafeFreeze
by way of runSTUArray.
As runSTUArray makes it impossible to
use the mutable array after freezing, this encapsulated use
of unsafeFreeze is always safe.

An expression, such as, generate (1,100) ((^2) . fromIntegral)
produces an unboxed array with the first one hundred square
numbers.
Internally, it is based on the initialisation of a
mutable array, but this is completely abstracted over by the
definition of generate.
While there is no inbuilt need to ever
freeze a mutable array, good functional programming style requires
to keep mutable code as localised as possible and to avoid passing
mutable data structures around.

SUMMARY

Well written code based on unboxed arrays and using the discussed
pattern to create arrays by initialising a mutable version,
which is subsequently frozen, can achieve performance comparable
to low-level C code.
In fact, the collection-oriented high-performance
array frameworks that we will discuss in subsequent blog posts
work exactly in this ma


# いまここ

https://www.tweag.io/blog/2017-09-27-array-package/